# NeOkusam Kitap Öneri Sistemi

In [1]:
# Kitaplarla ilgili yapılan çalışmaları inceledik ve veri setleri araştırdık.
# Veri setlerini analiz ettik, model kurduk, test ettik.

# 2 veri setinde başarılı sonuçlara ulaştık:

# 1- GitHub - GoodBooks 10K / GoodReads veritabanından alınmış 10.000 kitaplık veri seti 
# 2- Kaggle - BookData / GoodReads veritabanından alınmış 54.300 kitaplık veri seti 

# Tavsiye Sistemimizin Altyapısı

# 1- İçerik Temelli Filtreleme
# 2- Kullanıcı Tabanlı İşbirlikçi Filtreleme
# 3- Model Tabanlı İşbirlikçi Filtreleme / Matris Çarpanlarına Ayırma


# Veri ön işleme ve özellik mühendisliği adımları ile, veri setimizi
# tavsiye sistemimizin filtereleme metodlarına yeterli ve doğru girdi sağlayacak son haline getirdik.


In [2]:
# Kitap Öneri Sistemimizin verimli çalışması ve doğru kitapların doğru okuyuculara ulaşabilmesi için 
# İlk iki kritik Adım:

# 1- Yeterli sayıda, doğru ve güvenirlir bilgi içeren bir kaynak bulmak: Doğru veri setini seçmek 
# 2- Sisteme en doğru girdiyi sağlayacak şekilde veriyi derlemek ve düzenlemek.


# Birlikte Veri Ön İşleme Adımlarımızı İnceleyelim:

In [3]:
# Önce kütüphaneleri ve veri dosyalarını yüklüyoruz.

In [4]:
import numpy as np
import pandas as pd

import seaborn as sns 
import matplotlib.pyplot as plt
# Plotly Libraris
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 50)
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Veri setlerimizi tanıyalım:

# 1- GitHub - GoodBooks 10K 

    # Veri seti 3 dosyadan oluşuyor: `ratings.csv`, `books.csv`, `book_tags.csv`, `tags.csv` 

    # `ratings.csv`: okuyucuların kitaplara verdiği puanları içerir. 
    #  Dosyada 53,424 kullanıcının, 10.000 kitap için atadığı 980.000 kullanıcı puanlaması yer alır.

    # `books.csv`: kitap adı, yazar, yayınlanma tarihi gibi kitap ile ilgili detaylı bilgiler içerir.

    # `tags.csv`: okuyucuların GoodReads platformunda kitaplara atadıkları kısa tanımlamalardan oluşan tag_name'leri 
    #  ve ilgili tag_id'leri içerir.

    # `book_tags`: kitap ve tag_id bazında atanan toplam tag adetlerini içerir.

# 2- Kaggle - BookData 

    # Veri seti tek dosyadan oluşuyor: 'books_data'
    
    # 'books_data': yazar, baskı, format, sayfa adedi, kullanıcı puanları ve kullanıcı yorumları 
    # gibi kitap ile ilgili detaylı bilgiler içerir.


# Veri setlerimizi yükleyelim

In [6]:
# 1- GitHub - GoodBooks 10K: `ratings.csv`, `books.csv`, `book_tags.csv`, `tags.csv` veri setlerini yüklüyoruz

In [7]:
# df_books = pd.read_csv('/Users/melisorhun/PycharmProjects/pythonProject/Book_Reccomendation/Final_Project_220922/Final_Proje/books.csv',index_col=0)
# df_ratings = pd.read_csv('/Users/melisorhun/PycharmProjects/pythonProject/Book_Reccomendation/Final_Project_220922/Final_Proje/ratings.csv',index_col=0)
# df_tags = pd.read_csv('/Users/melisorhun/PycharmProjects/pythonProject/Book_Reccomendation/Final_Project_220922/Final_Proje/tags.csv',index_col=0)
# df_book_tags = pd.read_csv('/Users/melisorhun/PycharmProjects/pythonProject/Book_Reccomendation/Final_Project_220922/Final_Proje/book_tags.csv',index_col=0)
# df = pd.read_csv('/Users/melisorhun/PycharmProjects/pythonProject/Book_Reccomendation/Final_Project_220922/Final_Proje/book_data.csv',index_col=0)


# df_books=df_books.reset_index()
# df_ratings=df_ratings.reset_index()
# df_read=df_read.reset_index()
# df_tags = df_tags.reset_index()
# df_book_tags_copy=df_book_tags.copy()
# df_book_tags=df_book_tags.reset_index()

## 2- Kaggle - BookData: 'book_data' veri setini yüklüyoruz.
# df=df.reset_index()

In [80]:
df_books = pd.read_csv('books.csv',index_col=0)
df_books=df_books.reset_index()

df_ratings = pd.read_csv('ratings.csv',index_col=0)
df_ratings=df_ratings.reset_index()

df_read = pd.read_csv('to_read.csv',index_col=0)
df_read=df_read.reset_index()

df_tags = pd.read_csv('tags.csv',index_col=0)
df_tags = df_tags.reset_index()

df_book_tags = pd.read_csv('book_tags.csv',index_col=0)
df_book_tags_copy=df_book_tags.copy()
df_book_tags=df_book_tags.reset_index()

## 2- Kaggle - BookData: 'book_data' veri setini yüklüyoruz.
df = pd.read_csv('book_data.csv',index_col=0)
df=df.reset_index()

In [81]:
# Veri setlerimizin boyutlarına bakalım

In [82]:
print("df_books.shape     :",df_books.shape)
print("df_ratings.shape   :",df_ratings.shape)
print("df_read.shape      :",df_read.shape)
print("df_tags.shape      :",df_tags.shape)
print("df_book_tags.shape :",df_book_tags.shape)
print("df.shape           :",df.shape)

df_books.shape     : (10000, 23)
df_ratings.shape   : (5976479, 3)
df_read.shape      : (912705, 2)
df_tags.shape      : (34252, 2)
df_book_tags.shape : (999912, 3)
df.shape           : (54301, 12)


In [11]:
# df_ratings: Yüksek sayıda rating olduğunu görüyoruz, çalışmamız için zengin bir kaynak. 

In [12]:
# Veri setlerimize bir göz atalım

In [13]:
df_books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [14]:
df_ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [15]:
df_read.head()

,user_id,book_id
0,9,8
1,15,398
2,15,275
3,37,7173
4,34,380


In [16]:
df_tags.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [17]:
df_book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [18]:
# df_book_tags ve df_tags'ı ortak değişkenleri tag_id'yi kullanarak birleştirelim
df_merge_tags=df_book_tags.merge(df_tags,on="tag_id")
df_merge_tags

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
1,2,30574,24549,to-read
2,3,30574,496107,to-read
3,5,30574,11909,to-read
4,6,30574,298,to-read
...,...,...,...,...
999907,31538635,14690,6,hogwarts
999908,32848471,16149,21,jan-2017
999909,33288638,27821,9,single-mom
999910,33288638,11478,7,fave-author


In [19]:
df_merge_tags["goodreads_book_id"].value_counts().sort_values()

# Çoğu kitap için 100 farklı tag bulunmakta olduğunu görüyoruz

22931009     56
18607805     62
280958       94
33          100
3           100
           ... 
18655937    100
23503361    100
24473465    100
16093188    100
395233      100
Name: goodreads_book_id, Length: 10000, dtype: int64

In [20]:
df_merge_tags[df_merge_tags["goodreads_book_id"]==33288638].iloc[:20]

# Tag tanımlarını ve tag adetlerini inceliyoruz.

,goodreads_book_id,tag_id,count,tag_name
9982,33288638,30574,14116,to-read
24122,33288638,11557,354,favorites
33898,33288638,8717,1196,currently-reading
46625,33288638,11743,21,fiction
56456,33288638,5207,7,books-i-own
80658,33288638,11590,48,favourites
102078,33288638,6857,14,children
110059,33288638,2104,10,all-time-favorites
143913,33288638,3389,9,audiobook
146512,33288638,1128,32,5-stars


In [21]:
# Kitap bazında ortalama 597 rating olduğunu görüyoruz. Medyan değeri 248.

In [22]:
df_rating_count=df_ratings.groupby("book_id").agg({"user_id":"count"})
df_rating_count.columns=["count"]
df_rating_count["count"].mean()

597.6479

In [23]:
df_rating_count["count"].median()

248.0

# İngilizce olmayan kitapları veri setinden çıkaralım.

In [24]:
df_books.language_code.value_counts()

# İngilizce olmayan çok az sayıda kitap var, bu kitapları veri setinden çıkarıyoruz.
# language_code bilgisi ingilizce olmayan 120 civarında kitabı siliyoruz.

eng      6341
en-US    2070
en-GB     257
ara        64
en-CA      58
fre        25
ind        21
spa        20
ger        13
per         7
jpn         7
por         6
pol         6
en          4
nor         3
dan         3
fil         2
ita         2
vie         1
tur         1
nl          1
swe         1
rum         1
mul         1
rus         1
Name: language_code, dtype: int64

In [25]:
df_books=df_books[(df_books.language_code.isin(["eng","en-US","en-GB","en-CA"]))| (df_books.language_code.isna()) ]
df_books.language_code.value_counts()

eng      6341
en-US    2070
en-GB     257
en-CA      58
Name: language_code, dtype: int64

In [26]:
df_books.shape

(9810, 23)

# Missing Values - Eksik değerlere anlamlı değerler atayalım.

In [27]:
# df_books["original_title"] bilgisi eksik olan alanların doldurulması:

# Bir kitap serisinin bir parçası olan kitapların isimleri "original_title" altında yer alırken ait oldukları 
# kitap serisi, 'title' başlığı altında yer almaktadır.

# Örneğin: Harry Potter ve Felsefe Taşı için kitap adı "original_title" başlığı altında, 
# Harry Potter Serisi'nin ilk kitabı olduğu bilgisi 'title' başlığı altında yer almaktadır.

# Veri setimizde "original_title" bilgisi NAN olan 573 değeri, 'title' bilgisini atayarak dolduruyoruz.

In [28]:
df_books[df_books["original_title"].isna()]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
74,75,227443,227443,3185154,193,014028009X,9.780140e+12,Helen Fielding,1996.0,NaN,"Bridget Jones's Diary (Bridget Jones, #1)",eng,3.75,736498,754498,8157,28973,59309,198555,254836,212825,https://images.gr-assets.com/books/1292060045m...,https://images.gr-assets.com/books/1292060045s...
142,143,18143977,18143977,25491300,139,1476746583,9.781477e+12,Anthony Doerr,2014.0,NaN,All the Light We Cannot See,eng,4.31,470001,547827,53413,6209,14527,61020,185239,280832,https://images.gr-assets.com/books/1451445646m...,https://images.gr-assets.com/books/1451445646s...
209,210,345627,345627,335933,132,159514174X,9.781595e+12,Richelle Mead,2007.0,NaN,"Vampire Academy (Vampire Academy, #1)",en-US,4.14,248283,455151,19720,12535,22043,73357,129382,217834,https://images.gr-assets.com/books/1361098973m...,https://images.gr-assets.com/books/1361098973s...
214,215,9969571,9969571,14863741,112,030788743X,9.780308e+12,Ernest Cline,2011.0,NaN,Ready Player One,eng,4.31,349423,421819,52120,5150,12928,48285,134107,221349,https://images.gr-assets.com/books/1500930947m...,https://images.gr-assets.com/books/1500930947s...
256,257,13023,13023,2933712,4,517223627,9.780517e+12,"Jane Carruth, Lewis Carroll, Rene Cloke",1865.0,NaN,Alice in Wonderland,eng,4.03,343252,343271,4876,7467,18101,70170,109730,137803,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9942,9943,653559,653559,14738632,5,866114262,9.780866e+12,"Malvina G. Vogel, Arthur Conan Doyle",1977.0,NaN,Sherlock Holmes and the Case of the Hound of t...,eng,4.49,9805,9861,28,55,91,830,2900,5985,https://images.gr-assets.com/books/1292693979m...,https://images.gr-assets.com/books/1292693979s...
9944,9945,18943009,18943009,25364368,10,NaN,NaN,R.K. Lilley,2013.0,NaN,"Rock Bottom (Tristan & Danika, #2)",eng,4.46,18490,18709,1562,109,345,1710,5238,11307,https://images.gr-assets.com/books/1385363364m...,https://images.gr-assets.com/books/1385363364s...
9956,9957,6332526,6332526,6518250,36,1416583408,9.781417e+12,Alexandra Horowitz,2009.0,NaN,"Inside of a Dog: What Dogs See, Smell, and Know",en-GB,3.55,8770,10386,1611,324,1143,3452,3447,2020,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9957,9958,7494887,7494887,9646479,31,045123023X,9.780451e+12,Hugh Ambrose,2009.0,NaN,The Pacific,eng,4.09,9035,9292,295,168,390,1716,3148,3870,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


In [29]:
df2=df_books[df_books["original_title"].isna()]
for index in df2.index:
    try:
        title=df2.loc[index,"title"]
        book_title=title.split("(")[0]
        if len(book_title)>1:
            df_books.loc[index,["original_title"]]=book_title
    except:
        df_books.loc[index,["original_title"]]=title


In [30]:
df_books[df_books["original_title"].isna()]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url


# Çoklama değerleri silelim.

In [31]:
df_books.shape

(9810, 23)

In [32]:
df_books=df_books.loc[df_books[["authors","original_title"]].drop_duplicates().index,:]
df_books.shape

# Duplike kayıtları siliyoruz. 

(9800, 23)

In [33]:
df_books = df_books.rename(columns={'authors': 'book_authors', 'original_title': 'book_title'})

# Veri setimizde aynı kitap için birden fazla kitap özeti (description) yer alıyor.
# Kitap özetlerini tek bir kolonda birleştirebilmek için kolon isimlerini yeniden adlandırıyoruz.

In [34]:
df_books.head(1)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,book_authors,original_publication_year,book_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...


# Bir kitaba ait farklı özetleri tek bir başlık altında birleştirelim.

In [35]:
# birden fazla decrip olanları bulduk desc olmayanları eledik. Birden fazla description'ı olanların descriptionları
# tek bir başlık altında topluyoruz.. Böylelikle ilgili kitaplar için zengin bir içerik verisine sahip oluyoruz.

In [36]:
dff=df[df[["book_authors","book_title"]].duplicated()]
dff["book_desc30"]=dff["book_desc"].str[:30]
dff=dff.drop_duplicates("book_desc30")
dff=dff[dff.book_desc.notna()]
dff.shape

# birden fazla kitap özeti (decription) olan kitapları buluyoruz, kitap özeti olmayan kitapları eliyoruz.
# Her kitap için, o kitaba ait tüm description'ları 'book_desc' başlığı altında topluyoruz. 
# İçerik temelli filtreleme ile kitap tavsiyesi oluşturmak için zengin bir içerik elde ediyoruz.

(2437, 13)

In [37]:
changed_books=[]
for index in dff.index:
    book_title=dff.loc[index,"book_title"]
    book_authors=dff.loc[index,"book_authors"]
    dfx=dff[dff.book_title==book_title][dff.book_authors==book_authors]
    if len(dfx)>1:
        #print(dfx[["book_authors","book_title","book_desc"]])
        all_text_together=dfx["book_desc"].values.sum()
        df.loc[dfx.index,"book_desc"]=all_text_together
        changed_books.append(list(dfx.index))

In [38]:
dff.loc[changed_books[0],["book_title","book_authors","book_desc"]]

,book_title,book_authors,book_desc
137,Dune,Frank Herbert,Here is the novel that will be forever conside...
31351,Dune,Frank Herbert,"Before The Matrix, before Star Wars, before En..."
34547,Dune,Frank Herbert,The Duke of Atreides has been manoeuvred by hi...


In [39]:
dff.loc[changed_books[0],"book_desc"].iloc[1]

"Before The Matrix, before Star Wars, before Ender's Game and Neuromancer, there was Dune: winner of the prestigious Hugo and Nebula awards, and widely considered one of the greatest science fiction novels ever written.Melange, or 'spice', is the most valuable - and rarest - element in the universe; a drug that does everything from increasing a person's life-span to making intersteller travel possible. And it can only be found on a single planet: the inhospitable desert world Arrakis.Whoever controls Arrakis controls the spice. And whoever controls the spice controls the universe.When the Emperor transfers stewardship of Arrakis from the noble House Harkonnen to House Atreides, the Harkonnens fight back, murdering Duke Leto Atreides. Paul, his son, and Lady Jessica, his concubine, flee into the desert. On the point of death, they are rescued by a band for Fremen, the native people of Arrakis, who control Arrakis' second great resource: the giant worms that burrow beneath the burning de

In [40]:
df.loc[changed_books[0],"book_desc"].iloc[1]

"Here is the novel that will be forever considered a triumph of the imagination. Set on the desert planet Arrakis, Dune is the story of the boy Paul Atreides, who would become the mysterious man known as Muad'Dib. He would avenge the traitorous plot against his noble family--and would bring to fruition humankind's most ancient and unattainable dream.A stunning blend of adventure and mysticism, environmentalism and politics, Dune won the first Nebula Award, shared the Hugo Award, and formed the basis of what it undoubtedly the grandest epic in science fiction.Before The Matrix, before Star Wars, before Ender's Game and Neuromancer, there was Dune: winner of the prestigious Hugo and Nebula awards, and widely considered one of the greatest science fiction novels ever written.Melange, or 'spice', is the most valuable - and rarest - element in the universe; a drug that does everything from increasing a person's life-span to making intersteller travel possible. And it can only be found on a 

In [41]:
df.shape

(54301, 12)

In [42]:
df=df.loc[df[["book_authors","book_title"]].drop_duplicates().index,:]
df.shape

# Aynı kitap ve yazar ismine sahip  çoklamaları siliyoruz.

(51527, 12)

# Tüm verileri df isimli yeni bir dataframe'de birleştirelim.

In [43]:
df=df_books.merge(df, how='left', on=['book_authors','book_title'])
df

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,book_authors,original_publication_year,book_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url_x,small_image_url,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,genres,image_url_y
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135.0,160706.0,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,About three things I was absolutely positive.F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268.0,97991.0,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197.0,79450.0,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,Alternate Cover Edition ISBN: 0743273567 (ISBN...,US / CAN,Paperback,NaN,180 pages,3.90,3141842.0,56953.0,Classics|Fiction|Academic|School|Literature|Hi...,https://images.gr-assets.com/books/1490528560l...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,"Bayou Moon (The Edge, #2)",eng,4.09,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...,"The Edge lies between worlds, on the border be...",NaN,Mass Market Paperback,9.78044E+12,447 pages,4.10,21145.0,1310.0,Fantasy|Urban Fantasy|Fantasy|Romance|Fantasy|...,https://images.gr-assets.com/books/1307445460l...
9796,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,"Means of Ascent (The Years of Lyndon Johnson, #2)",eng,4.25,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9797,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,The Mauritius Command,eng,4.35,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/14553735

In [44]:
df.isna().sum()

book_id                         0
goodreads_book_id               0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          623
isbn13                        537
book_authors                    0
original_publication_year      20
book_title                      0
title                           0
language_code                1083
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url_x                     0
small_image_url                 0
book_desc                    4766
book_edition                 9268
book_format                  4773
book_isbn                    5098
book_pages                   4762
book_rating                  4757
book_rating_co

In [45]:
# Veri setimizde 'language_code''u NAN olan ve dili ingilizce olmayan kitaplar olduğunu farkediyoruz.
# Hedefimiz sadece ingilizce dilinde kitaplar önermek olduğu için bu kitapları bulup veri setinden siliyoruz.

indexs=[]

df2=df[df.language_code.isna()]
for index in df2.index:
    try:
        desc=df2.loc[index,"book_desc"]
        counter=0
        for val in ["is","are","will","was","had","have","the","The","and"]:
            if val not in desc.split():
                counter+=1
            else:
                break
        if counter==9:
            print("index",index,desc[:100],"*"*10,"\n")
            indexs.append(index)
    except:
        pass

index 2802 Barcelona, 1957. Daniel Sempere y su amigo Fermín, los héroes de La Sombra del Viento, regresan de n ********** 

index 4997 na hermosa joven se ha enamorado de su profesor de literatura, que es un hombre casado. Cuando éste  ********** 

index 6129 Diez años después de Trainspotting, Sick Boy está en franca decadencia. Acaba de dejar su piso en un ********** 

index 6601 Rare book ********** 

index 8535 Edición argentina, tomo 1 de 3. ********** 

index 8941 Page considera que su vida es perfecta: madre de dos hijos encantadores, casada con un hombre afectu ********** 



In [46]:
df=df[~(df.index.isin(indexs))]
df

# ingilizce olmayan kitapların index bilgilerini belirleyip, bu kitaplar harici kitapları seçerek df'i güncelliyoruz.
# [2802, 4997, 6129, 6601, 8535, 894

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,book_authors,original_publication_year,book_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url_x,small_image_url,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,genres,image_url_y
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135.0,160706.0,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,About three things I was absolutely positive.F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268.0,97991.0,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197.0,79450.0,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,Alternate Cover Edition ISBN: 0743273567 (ISBN...,US / CAN,Paperback,NaN,180 pages,3.90,3141842.0,56953.0,Classics|Fiction|Academic|School|Literature|Hi...,https://images.gr-assets.com/books/1490528560l...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,"Bayou Moon (The Edge, #2)",eng,4.09,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...,"The Edge lies between worlds, on the border be...",NaN,Mass Market Paperback,9.78044E+12,447 pages,4.10,21145.0,1310.0,Fantasy|Urban Fantasy|Fantasy|Romance|Fantasy|...,https://images.gr-assets.com/books/1307445460l...
9796,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,"Means of Ascent (The Years of Lyndon Johnson, #2)",eng,4.25,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9797,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,The Mauritius Command,eng,4.35,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/14553735

In [47]:
indexs=df[df["book_pages"].notna()].index
for index in indexs:
    val=df.loc[index,"book_pages"]
    val=int(val.split()[0])
    df.loc[index,["book_pages"]]=val
    
# Kitap boyutu, 'book_pages' adı altında, veri tipi object olarak yer almaktadır. ör: 374 pages
# Veriyi kategorik değişkene çevirmek için sayı değerlerini ayrıştırıp integer olarak tanımlıyoruz. ör: 374

In [48]:
df["book_pages"].values[:10]

array([374, nan, 498, 324, 180, 313, 319, 277, nan, 432], dtype=object)


# Özellik Mühendisliği - Feature Engineering

In [49]:
# 'Applied machine learning is basically feature engineering' - Andrew Ng
#  (Uygulamalı makine öğrenmesi temel olarak özellik mühendisliğidir.)


# Hedefimiz;
# Tavsiye sistemlerimizin verimli ve doğru çalışması için;
        
            # Kitapların özelliklerini iyi ifade eden
            # Okuyucunun ilgi ve beğenilerini iyi yansıtan
		                                                  # yeni ve anlamlı değişkenler türetmek.

# Yeni Oluşturduğumuz değişkenlerimiz:
# 
# book_series: serisi olan kitapların, ait olduğu seri isimlerinden oluşturduk.
# rating_count, review_count: ekik değerleri, iki dosya arasında oransal formüller kurularak güncelledik 
# book_rating_obj: kitapları 0.5 hassasiyet ile rating değerlerine ayırdık.
# book_score: kitap bazında puanları en gerçekçi yansıtan değerlere ulaşmak için matematiksel bir formül geliştirdik.

# İçerik Temelli Tavsiye Sistemi için türettiğimiz değişkenler:
# tags: her kitap için atanmış ~100 farklı tag içeriğini uygun forma getirerek 'tags' başlığı altında birleştirdik.
# FE_text: book_title, book_authors, genres ve language_code değişken içeriklerinin birleşiminden oluşur.
# book_desc_tags_FE: book_desc,tags, FE_text olmak üzere dataframe içindeki tüm değişkenlerin içeriklerinin birleşiminden oluşur.  





In [50]:
# Kitap sayfa sayılarına göre rating, rating_count ve review_count durumları aşağıdaki gibi değişiklik göstermektedir.
# İncelememiz sonucunda şu ayrıntılar gözümüze çarpmaktadır.
# 1. 200-500 arası kitaplar toplam kitapların %75'ine takabül etmektedir.
# 2. kitap sayfa sayısı arttıkça kitapların beğenilme oranları artmaktadır.
# 3. kitap sayfa sayısı ortalamanın altında olan kitaplarda rating_count ve review_count değerleri düşüktür.

# book_pages_obj kategorik değişkenini oluşturalım

In [51]:
df["book_pages_obj"]=df["book_pages"]//100
df.loc[df[df["book_pages_obj"]>10].index,"book_pages_obj"]=10
dfx=df.groupby(["book_pages_obj"]).agg({"average_rating":["count","mean"],"book_rating_count":"mean","book_review_count":"mean"})
dfx.index=["0-100","100-200","200-300","300-400","400-500","500-600","600-700","700-800","800-900","900-1000","1000+"]
dfx.columns=["count_rate(%)","rating_mean","rating_count_mean","review_count_mean"]
dfx["count_rate(%)"]=(dfx["count_rate(%)"]/len(df[df["book_pages"].notna()])*100).round(1)
dfx["rating_mean"]=dfx["rating_mean"].round(2)
dfx[["rating_count_mean","review_count_mean"]]=dfx[["rating_count_mean","review_count_mean"]].astype(int)
dfx.index.names = ['page_intervals']
dfx

,count_rate(%),rating_mean,rating_count_mean,review_count_mean
page_intervals,,,,
0-100,2.7,4.11,60682,1921
100-200,6.7,3.96,73880,3063
200-300,20.7,3.93,71485,3798
300-400,35.0,3.98,79381,4592
400-500,19.2,4.01,83936,4442
500-600,7.5,4.05,93387,5086
600-700,3.5,4.08,78534,3907
700-800,1.6,4.05,88489,4330
800-900,1.0,4.12,94840,4375



# tags değişkenini oluşturalım (İçerik Temelli Tavsiye Sistemi için)

In [52]:
df_merge_tags=df_book_tags.merge(df_tags,on="tag_id")
df_merge_tags["tag_name"]=df_merge_tags["tag_name"]+" "

df_merge_tags["tags"]=""
tags=df_merge_tags.groupby(['goodreads_book_id']).agg({"tag_name": "sum"})

df_book_tags = df_book_tags_copy.copy()
df_book_tags["tags"]=""
df_book_tags.loc[tags.index,"tags"]=tags["tag_name"]
df_book_tags=df_book_tags.reset_index()
df_book_tags

# Kitap bazında atanmış olan tüm tag tag içeriklerini, 'tags' başlığı altında birleştirdik.
# İçerik tabanlı tavsiye sistemine zengin ve anlamlı bir girdi sağladık.

,goodreads_book_id,tag_id,count,tags
0,1,30574,167697,to-read fantasy favorites currently-reading yo...
1,1,11305,37174,to-read fantasy favorites currently-reading yo...
2,1,11557,34173,to-read fantasy favorites currently-reading yo...
3,1,8717,12986,to-read fantasy favorites currently-reading yo...
4,1,33114,12716,to-read fantasy favorites currently-reading yo...
...,...,...,...,...
999907,33288638,21303,7,to-read favorites currently-reading fiction bo...
999908,33288638,17271,7,to-read favorites currently-reading fiction bo...
999909,33288638,1126,7,to-read favorites currently-reading fiction bo...
999910,33288638,11478,7,to-read favorites currently-reading fiction bo...


In [53]:
# İncelememiz sonucu bazı taglerin("to-read","own-it", etc.) içerik olarak anlam taşımadığını görsekte,
# birçok tag'in içeriğe ve kitaba dair önemli bilgiler("harry-potter-series","childrens-books","kids",etc.) taşıdığını gördük.
# Bu tag'ler ile İçerik Temelli Tavsiye yapabileceğimizi farkettik.

In [54]:
df_book_tags.loc[0,"tags"]

'to-read fantasy favorites currently-reading young-adult fiction harry-potter books-i-own owned ya series favourites magic childrens owned-books re-read adventure children j-k-rowling children-s sci-fi-fantasy childhood all-time-favorites default my-books classics reread i-own audiobook 5-stars children-s-books favorite-books kids novels fantasy-sci-fi favorite middle-grade audiobooks paranormal read-more-than-once my-library ya-fantasy teen witches english urban-fantasy british jk-rowling books read-in-2016 supernatural re-reads mystery ya-fiction harry-potter-series my-favorites own-it childrens-books library audio young-adult-fiction novel 2005 scifi-fantasy wizards faves favorite-series read-in-2015 made-me-cry juvenile shelfari-favorites kindle youth romance favourite to-buy read-in-2014 to-re-read all-time-favourites childhood-favorites kids-books ebook rereads contemporary read-in-english 5-star coming-of-age science-fiction-fantasy read-in-2017 england children-s-literature fav

In [55]:
# Oluşturduğumuz "tags" isimli değişkeni temel dataframe'mimize merge ile ekledik.

In [56]:
df4=df.merge(df_book_tags[["goodreads_book_id","tags"]].drop_duplicates(),on="goodreads_book_id")
df4.head(2)
# tags değişkenini df veri setimize ekliyoruz ve df4 oluşturuyoruz, çoklu kayıtları siliyoruz.

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,book_authors,original_publication_year,book_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url_x,small_image_url,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,genres,image_url_y,book_pages_obj,tags
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374,4.33,5519135.0,160706.0,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...,3,to-read fantasy favorites currently-reading yo...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,to-read fantasy favorites currently-reading yo...


# FE_text değişkenini oluşturalım  (İçerik Temelli Tavsiye Sistemi için)

In [57]:
# FE_text değişkeni; book_title, book_authors, genres ve language_code değişken içeriklerinin birleşiminden oluşur.

# Genres değişkeni kitap içeriğine dair kategori/tür bilgisi taşır.
# Okuyucuların hoşlandığı ve hoşlanmadığı kitap türleri (genres) vardır.
# Bir yazar pek çok farklı konuda kitap yazmış olsa da, o yazara ait kitapların kendine özgü ayırt edici bir hissi, 
# bir lezzeti olur. Bu lezzeti bize kelimeleri ile hissettirir. 
# Bu nedenle yazar ismi içeriğe dair önemli bilgiler taşır.

# Yazarlar kitaplarını adlandırırken, o kitabı en iyi tanımlayacak ifadeyi seçmeye özen gösterirler. 
# Bu nedenle kitap ismi de içeriğe dair önemli bilgiler taşır.

# Bir kullanıcı birden çok dil bilse de belirli bir dilde yazılan kitapları daha çok tercih ediyor olabilir. 
# Bu kişinin, dil tercihini de dikkate alarak tavsiyede bulunabilmek için bu alanı da değişkenimize ekliyoruz.

# Böylelikle, 'İçerik Temelli Tavsiye Sistemimiz''e yeni bir anlamlı değişken kazandırmış olduk.

In [58]:
df4["FE_text"]=" "

for index in df4.index:
    #print(index)
    text=[]
    try:
        text+=df4.loc[index,"book_title"].replace("(","").replace(")","").replace(" ","-").split()
    except:
        pass
    try:
        text+=[df4.loc[index,"book_authors"].replace(" ","-")]
    except:
        pass
    try:
        text+=df4.loc[index,"genres"].replace(" ","-").split("|")
    except:
        pass
    try:
        text+=[df4.loc[index,"language_code"]]
    except:
        pass

    
    text2=""
    for i in text:
        if type(i)!=float:
            text2+=i+" "
    text2=text2.strip()
    
    df4.loc[index,"FE_text"] =text2   

# book_desc_tags_FE değişkenini oluşturalım (İçerik Temelli Tavsiye Sistemi için)

In [59]:
# Tüm değişkenlerin içeriklerini bir araya getirerek, İçerik Temelli Tavsiye Sistemi için en kapsamlı değişkenimizi oluşturduk.

# Burada en can alıcı soru:
# 'Bu değişkeni oluşturduktan sonra diğer değişkenleri neden drop etmediniz?' olacaktır.

# Cevabımız: 
# Bilindiği gibi tag'lerde "to-read","own-it" gibi içeriği temsil etmeyen kelimeler mevcuttur.
# Bu kelimeler içerik temelli tavsiye oluştururken, sistemi kayda değer şekilde olumsuz etkileyebilmektedir.
# Bu kelimelere tek tek bakıp silmek bir çözüm olsa da tüm veriyi bu şekilde temizlemek mümkün değildir.

# Diger degiskenlerle ayrı ayrı İçerik Temelli Tavsiyeler oluşturarak, burada çıkan gürültülü sonucu diğer veri setleri sonuçlarını ile 
# ilusturdugumuz belli bir katsayı çarpanı ile  toplamamiz sayesinde bu gürültüyü gözardı edebileceğimiz  minumum değere indirgeyecegiz.

# Ancak biz buna gerek kalmadan, diğer değişkenler ile ayrı ayrı İçerik Temelli Tavsiye yaparak 
# bu etkiyi minumuma indirgediyeceğiz. 



In [60]:
df4['book_desc'] = df4['book_desc'].fillna('')
df4["book_desc_tags_FE"]=(df4["book_desc"]+" "+df4["tags"]+" "+df4["FE_text"])
df4[["book_desc","tags","FE_text","book_desc_tags_FE"]]

,book_desc,tags,FE_text,book_desc_tags_FE
0,Winning will make you famous. Losing means cer...,to-read fantasy favorites currently-reading yo...,The-Hunger-Games Suzanne-Collins Young-Adult F...,Winning will make you famous. Losing means cer...
1,,to-read fantasy favorites currently-reading yo...,Harry-Potter-and-the-Philosopher's-Stone J.K.-...,to-read fantasy favorites currently-reading y...
2,About three things I was absolutely positive.F...,to-read fantasy favorites currently-reading yo...,Twilight Stephenie-Meyer Young-Adult Fantasy R...,About three things I was absolutely positive.F...
3,The unforgettable novel of a childhood in a sl...,to-read favorites currently-reading young-adul...,To-Kill-a-Mockingbird Harper-Lee Classics Fict...,The unforgettable novel of a childhood in a sl...
4,Alternate Cover Edition ISBN: 0743273567 (ISBN...,to-read favorites currently-reading young-adul...,The-Great-Gatsby F.-Scott-Fitzgerald Classics ...,Alternate Cover Edition ISBN: 0743273567 (ISBN...
...,...,...,...,...
9789,"The Edge lies between worlds, on the border be...",to-read fantasy favorites currently-reading fi...,Bayou-Moon Ilona-Andrews Fantasy Urban-Fantasy...,"The Edge lies between worlds, on the border be..."
9790,,to-read favorites currently-reading books-i-ow...,Means-of-Ascent- Robert-A.-Caro eng,to-read favorites currently-reading books-i-o...
9791,"""O'Brian's Aubrey-Maturin volumes actually con...",to-read favorites currently-reading fiction bo...,The-Mauritius-Command Patrick-O'Brian Historic...,"""O'Brian's Aubrey-Maturin volumes actually con..."
9792,The acclaimed author of the groundbreaking bes...,to-read favorites currently-reading books-i-ow...,Cinderella-Ate-My-Daughter:-Dispatches-from-th...,The acclaimed author of the groundbreaking bes...


In [61]:
# oluşturulan book_desc_tags_FE değişkenine dair bir örnek

In [62]:
df4.loc[0,"book_desc_tags_FE"]

"Winning will make you famous. Losing means certain death.The nation of Panem, formed from a post-apocalyptic North America, is a country that consists of a wealthy Capitol region surrounded by 12 poorer districts. Early in its history, a rebellion led by a 13th district against the Capitol resulted in its destruction and the creation of an annual televised event known as the Hunger Games. In punishment, and as a reminder of the power and grace of the Capitol, each district must yield one boy and one girl between the ages of 12 and 18 through a lottery system to participate in the games. The 'tributes' are chosen during the annual Reaping and are forced to fight to the death, leaving only one survivor to claim victory.When 16-year-old Katniss's young sister, Prim, is selected as District 12's female representative, Katniss volunteers to take her place. She and her male counterpart Peeta, are pitted against bigger, stronger representatives, some of whom have trained for this their whole

# rating_count, review_count için eksik değerleri düzenleyelim

In [63]:
# Biz bu çalışmamızda iki farklı veri setini birleştirerek çok daha kapsamlı veri seti elde ettik.
# Bu birleştirmeyi, yazar adı ile kitap adı aynı olan gözlemleri birleştirerek yaptık.
# Bu iki veri setinde farklı değişkenler olduğu gibi aynı anlamı taşıyan değişkenlerde mevcut.
# bir veri setinde   'book_rating', 'book_rating_count', 'book_review_count'
# diğer veri setinde 'average_rating','ratings_count','work_text_reviews_count'
# adlı aynı anlamı taşıyan değişkenler mevcut. 

# Burada değerlere baktığımızda ilk değişkenler diğerlerine göre %45 oranında daha fazla bilgi taşımakta.
# Bu nedenle ilk değişkenleri seçmemiz mantıklı.

# Ancak bu değişkenlerde NAN değerleri mevcut. Bu NAN değerlerini ikinci değişkenleri farkları oranı ile(yaklaşık %45) ile
# çarparak aynı anlamı taşıyan en iyi değişkeni elde etmiş olduk.

In [64]:
df4["book_rating"].mean(),df4["book_rating"].mean()

(4.00202104427237, 4.00202104427237)

In [65]:
df4["book_rating_count"].mean(),df4["ratings_count"].mean()

(79341.21540599564, 54641.59301613233)

In [66]:
df4["book_review_count"].mean(),df4["work_text_reviews_count"].mean()

(4208.718681755013, 2946.636614253625)

In [67]:
df4.loc[df4[df4["book_rating"].isna()].index,"book_rating"]=df4[df4["book_rating"].isna()]["average_rating"].values
df4.loc[df4[df4["book_rating_count"].isna()].index,"book_rating_count"]=((df4[df4["book_rating_count"].isna()]["ratings_count"].values)*(df4["book_rating_count"].mean()/df4["ratings_count"].mean()))
df4.loc[df4[df4["book_review_count"].isna()].index,"book_review_count"]=((df4[df4["book_review_count"].isna()]["work_text_reviews_count"].values)*(df4["book_review_count"].mean()/df4["work_text_reviews_count"].mean()))
df4[['book_rating_count', 'book_review_count','ratings_count','work_text_reviews_count']]=df4[['book_rating_count', 'book_review_count','ratings_count','work_text_reviews_count']].astype(int)
df4[['book_rating','book_rating_count', 'book_review_count','average_rating','ratings_count','work_text_reviews_count']]

,book_rating,book_rating_count,book_review_count,average_rating,ratings_count,work_text_reviews_count
0,4.33,5519135,160706,4.34,4780653,155254
1,4.44,6682936,108361,4.44,4602479,75867
2,3.58,4281268,97991,3.57,3866839,95009
3,4.27,3745197,79450,4.25,3198671,72586
4,3.90,3141842,56953,3.89,2683664,51992
...,...,...,...,...,...,...
9789,4.10,21145,1310,4.09,17204,1180
9790,4.25,18269,564,4.25,12582,395
9791,4.35,11785,420,4.35,9421,374
9792,3.66,12811,2056,3.65,11279,1988


In [68]:
# Bu değişkenleri drop ediyoruz

In [69]:
df4=df4.drop(['average_rating','ratings_count','work_text_reviews_count'],axis=1)
df4.shape

(9794, 34)

# book_rating_obj değişkenini oluşturalım

In [ ]:
# Kitapları 0.5 hassasiyet ile rating değerlerine ayırıyoruz

In [70]:
col="book_rating"
new_col=col+"_obj"
df4[new_col]=0
for index in df4.index:
    val=df4.loc[index,col]
    if val<1.5:
        val2=1
    elif val<2:
        val2=1.5
    elif val<2.5:
        val2=2
    elif val<3:
        val2=2.5
    elif val<3.5:
        val2=3
    elif val<4:
        val2=3.5
    elif val<4.5:
        val2=4
    else:
        val2=5 

    df4.loc[index,[new_col]]=val2
df4[new_col].value_counts().sort_index()

2.0       1
2.5      12
3.0     298
3.5    4213
4.0    5133
5.0     137
Name: book_rating_obj, dtype: int64

# book_score değişkenini oluşturalım

In [ ]:
# Buradaki amacımız en çok okunan ve en iyi rating değeri alan kitaba en yüksek puan vermektir.
# Bir kitabın en çok okunduğunu book_rating_count ve book_review_count değişkenlerinden anlayabiliriz.

# Sadece book_rating değeri en yüksek olanı seçmememizin sebebi bir kitabı bir kişi okudu 
# ve 5 verdi ise o kitap ilk sırada çıkacaktır. Ama 10 bin kişinin okuduğu 4.99 ortalamalı kitap arkada kalacaktır.

# Bu durum mantıksız olduğu için bunu engelleyecek şekilde formülize ederek,
# score değişkeni ile en iyi kitablara en yüksek puanları vermiş olduk.

# Aşağıda bu formüle göre en başarılı kitapları başarı sırasına göre gösterdik.

In [71]:
new_col="book_score"
book_rating_max=5
book_rating_count_max=df4["book_rating_count"].max()
book_review_count_max=df4["book_review_count"].max()
df4[new_col]=0
for index in df4.index:
    book_rating=df4.loc[index,"book_rating"]
    book_rating_count=df4.loc[index,"book_rating_count"]
    book_review_count=df4.loc[index,"book_review_count"]
    ratings_1=df4.loc[index,"ratings_1"]
    ratings_5=df4.loc[index,"ratings_5"]
    
    book_rating/=book_rating_max
    book_rating_count/=book_rating_count_max
    book_review_count/=book_review_count_max
    
    book_score=(book_rating)*book_rating_count*book_review_count*((1000+ratings_5)/(1000+ratings_1))
    
    
    df4.loc[index,[new_col]]=book_score
df4["book_score"]=df4["book_score"]/df4["book_score"].max()
df4.sort_values("book_score",ascending=False).iloc[:15][["book_id","book_title","book_authors","book_score","book_rating","book_rating_count","book_review_count","ratings_1","ratings_5","genres"]]

,book_id,book_title,book_authors,book_score,book_rating,book_rating_count,book_review_count,ratings_1,ratings_5,genres
0,1,The Hunger Games,Suzanne Collins,1.000000,4.33,5519135,160706,66715,2706317,Young Adult|Fiction|Science Fiction|Dystopia|F...
1,2,Harry Potter and the Philosopher's Stone,"J.K. Rowling, Mary GrandPré",0.824569,4.44,6682936,108361,75504,3011543,NaN
24,25,Harry Potter and the Deathly Hallows,"J.K. Rowling, Mary GrandPré",0.719104,4.61,2536077,74189,9363,1318227,NaN
17,18,Harry Potter and the Prisoner of Azkaban,"J.K. Rowling, Mary GrandPré, Rufus Beck",0.665083,4.53,2661313,51560,6716,1266670,NaN
23,24,Harry Potter and the Goblet of Fire,"J.K. Rowling, Mary GrandPré",0.519502,4.53,2545470,44397,6676,1195045,NaN
16,17,Catching Fire,Suzanne Collins,0.461766,4.29,2127608,90972,10492,980309,Young Adult|Science Fiction|Dystopia|Fiction|F...
22,23,Harry Potter and the Chamber of Secrets,"J.K. Rowling, Mary GrandPré",0.413493,4.37,2583641,48808,8253,1065084,NaN
26,27,Harry Potter and the Half-Blood Prince,"J.K. Rowling, Mary GrandPré",0.396418,4.54,2437700,39307,7308,1161491,NaN
30,31,The Help,Kathryn Stockett,0.352735,4.46,1790905,80741,10235,942552,Fiction|Historical|Historical Fiction|Historic...
20,21,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré",0.320664,4.46,2519805,40971,9528,1124806,NaN


# book_series değişkenini oluşturalım

In [ ]:
# Bazı kitapların serileri mevcuttur. 
# Bu bilgileri "title" değişkeni içerisinden süzerek yeni bir değişken elde ettik.

In [72]:
new_col="book_series"
df4[new_col]="-"
for index in df4.index:
    try:
        title=df4.loc[index,"title"]
        series=title.split("(")[1].split(")")[0]
        if len(series)>1:
            df4.loc[index,[new_col]]=series
    except:
        pass
df4[["book_title","title","book_series"]]

,book_title,title,book_series
0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)","The Hunger Games, #1"
1,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,"Harry Potter, #1"
2,Twilight,"Twilight (Twilight, #1)","Twilight, #1"
3,To Kill a Mockingbird,To Kill a Mockingbird,-
4,The Great Gatsby,The Great Gatsby,-
...,...,...,...
9789,Bayou Moon,"Bayou Moon (The Edge, #2)","The Edge, #2"
9790,Means of Ascent,"Means of Ascent (The Years of Lyndon Johnson, #2)","The Years of Lyndon Johnson, #2"
9791,The Mauritius Command,The Mauritius Command,-
9792,Cinderella Ate My Daughter: Dispatches from th...,Cinderella Ate My Daughter: Dispatches from th...,-


In [ ]:
# Yeni oluşturulan değişkenler ile dataframe'mimiz karışık hal aldı.
# bu nedenle birbirine anlamca yakın olan değişkenleri bir araya getirdik.
# Okunması daha güzel bir df elde ettik.

In [73]:
df4.columns

Index(['book_id', 'goodreads_book_id',
       'best_book_id', 'work_id', 'books_count',
       'isbn', 'isbn13', 'book_authors',
       'original_publication_year', 'book_title',
       'title', 'language_code',
       'work_ratings_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4',
       'ratings_5', 'image_url_x',
       'small_image_url', 'book_desc',
       'book_edition', 'book_format',
       'book_isbn', 'book_pages', 'book_rating',
       'book_rating_count', 'book_review_count',
       'genres', 'image_url_y', 'book_pages_obj',
       'tags', 'FE_text', 'book_desc_tags_FE',
       'book_rating_obj', 'book_score',
       'book_series'],
      dtype='object')

In [75]:
new_cols=['book_id', 
       'book_title','book_series','title','book_authors', 'genres',
       'book_score', 'book_rating','book_rating_obj',
       'book_rating_count', 'book_review_count',
       'book_desc', 'tags','FE_text','book_desc_tags_FE',
       'ratings_1','ratings_2', 'ratings_3', 'ratings_4','ratings_5', 
       'book_edition', 'book_format','original_publication_year','language_code', 
       'book_pages','book_pages_obj', 'books_count', 
       'goodreads_book_id','book_isbn', 'isbn', 'isbn13',
       'image_url_x', 'image_url_y','small_image_url'
        
        ]
df5=df4[new_cols]
df5

,book_id,book_title,book_series,title,book_authors,genres,book_score,book_rating,book_rating_obj,book_rating_count,book_review_count,book_desc,tags,FE_text,book_desc_tags_FE,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,book_edition,book_format,original_publication_year,language_code,book_pages,book_pages_obj,books_count,goodreads_book_id,book_isbn,isbn,isbn13,image_url_x,image_url_y,small_image_url
0,1,The Hunger Games,"The Hunger Games, #1","The Hunger Games (The Hunger Games, #1)",Suzanne Collins,Young Adult|Fiction|Science Fiction|Dystopia|F...,1.000000e+00,4.33,4.0,5519135,160706,Winning will make you famous. Losing means cer...,to-read fantasy favorites currently-reading yo...,The-Hunger-Games Suzanne-Collins Young-Adult F...,Winning will make you famous. Losing means cer...,66715,127936,560092,1481305,2706317,NaN,Hardcover,2008.0,eng,374,3,272,2767052,9.78044E+12,439023483,9.780439e+12,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603l...,https://images.gr-assets.com/books/1447303603s...
1,2,Harry Potter and the Philosopher's Stone,"Harry Potter, #1",Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPré",NaN,8.245693e-01,4.44,4.0,6682936,108361,,to-read fantasy favorites currently-reading yo...,Harry-Potter-and-the-Philosopher's-Stone J.K.-...,to-read fantasy favorites currently-reading y...,75504,101676,455024,1156318,3011543,NaN,NaN,1997.0,eng,NaN,NaN,491,3,NaN,439554934,9.780440e+12,https://images.gr-assets.com/books/1474154022m...,NaN,https://images.gr-assets.com/books/1474154022s...
2,3,Twilight,"Twilight, #1","Twilight (Twilight, #1)",Stephenie Meyer,Young Adult|Fantasy|Romance|Paranormal|Vampire...,2.902011e-02,3.58,3.5,4281268,97991,About three things I was absolutely positive.F...,to-read fantasy favorites currently-reading yo...,Twilight Stephenie-Meyer Young-Adult Fantasy R...,About three things I was absolutely positive.F...,456191,436802,793319,875073,1355439,NaN,Paperback,2005.0,en-US,498,4,226,41865,9.78032E+12,316015849,9.780316e+12,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443l...,https://images.gr-assets.com/books/1361039443s...
3,4,To Kill a Mockingbird,-,To Kill a Mockingbird,Harper Lee,Classics|Fiction|Historical|Historical Fiction...,2.310594e-01,4.27,4.0,3745197,79450,The unforgettable novel of a childhood in a sl...,to-read favorites currently-reading young-adul...,To-Kill-a-Mockingbird Harper-Lee Classics Fict...,The unforgettable novel of a childhood in a sl...,60427,117415,446835,1001952,1714267,50th Anniversary,Paperback,1960.0,eng,324,3,487,2657,9.78006E+12,61120081,9.780061e+12,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680l...,https://images.gr-assets.com/books/1361975680s...
4,5,The Great Gatsby,-,The Great Gatsby,F. Scott Fitzgerald,Classics|Fiction|Academic|School|Literature|Hi...,4.942671e-02,3.90,3.5,3141842,56953,Alternate Cover Edition ISBN: 0743273567 (ISBN...,to-read favorites currently-reading young-adul...,The-Great-Gatsby F.-Scott-Fitzgerald Classics ...,Alternate Cover Edition ISBN: 0743273567 (ISBN...,86236,197621,606158,936012,947718,US / CAN,Paperback,1925.0,eng,180,1,1356,4671,NaN,743273567,9.780743e+12,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560l...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9789,9996,Bayou Moon,"The Edge, #2","Bayou Moon (The Edge, #2)",Ilona Andrews,Fantasy|Urban Fantasy|Fantasy|Romance|Fantasy|...,5.206229e-06,4.10,4.0,21145,1310,"The Edge lies between worlds, on the border be...",to-read fantasy favorites currently-reading fi...,Bayou-Moon Ilona-Andrews Fantasy Urban-Fantasy...,"The Edge lies between worlds, on the border be...",105,575,3538,7860,6778,NaN,Mass Market Paperback,2010.0,e

In [76]:
# Sonuç olarak,

# GoodBooks ve BookData veri setlerini ayrı ayrı veri ön işlemeden geçirdik.
# Temizleyip, düzenlediğimiz verileri setlerinin içeriği:
        # 1- Kitap ve kullanıcı puanlamaları: 10.000 kitap, 23 değişken
        # 2- Kitap özellik ve içerikleri: 54.301 kitap ,12 değişken

# 2 temel veri setini yazar ve kitap isimleri bazında eşleştirerek, yeni bir dataframe oluşturduk.
        # Bu dataframe, kitap ve yazar bazında tüm bilgi ve içerikleri taşıyan 9.794 kitap, 34 değişkenden oluşuyor. 
        # 6 milyon rating değeri ve her kitaba ait kitap özeti (description) içeriyor.
        
# Böylelikle;
# tüm tavsiye sistemlerine uygulayabileceğimiz, kitap bazında zengin içeriğe sahip, gelişmiş bir veri seti oluşturduk. 


,book_id,book_title,book_series,title,book_authors,genres,book_score,book_rating,book_rating_obj,book_rating_count,book_review_count,book_desc,tags,FE_text,book_desc_tags_FE,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,book_edition,book_format,original_publication_year,language_code,book_pages,book_pages_obj,books_count,goodreads_book_id,book_isbn,isbn,isbn13,image_url_x,image_url_y,small_image_url
0,1,The Hunger Games,"The Hunger Games, #1","The Hunger Games (The Hunger Games, #1)",Suzanne Collins,Young Adult|Fiction|Science Fiction|Dystopia|F...,1.000000,4.33,4.0,5519135,160706,Winning will make you famous. Losing means cer...,to-read fantasy favorites currently-reading yo...,The-Hunger-Games Suzanne-Collins Young-Adult F...,Winning will make you famous. Losing means cer...,66715,127936,560092,1481305,2706317,NaN,Hardcover,2008.0,eng,374,3,272,2767052,9.78044E+12,439023483,9.780439e+12,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603l...,https://images.gr-assets.com/books/1447303603s...
1,2,Harry Potter and the Philosopher's Stone,"Harry Potter, #1",Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPré",NaN,0.824569,4.44,4.0,6682936,108361,,to-read fantasy favorites currently-reading yo...,Harry-Potter-and-the-Philosopher's-Stone J.K.-...,to-read fantasy favorites currently-reading y...,75504,101676,455024,1156318,3011543,NaN,NaN,1997.0,eng,NaN,NaN,491,3,NaN,439554934,9.780440e+12,https://images.gr-assets.com/books/1474154022m...,NaN,https://images.gr-assets.com/books/1474154022s...
2,3,Twilight,"Twilight, #1","Twilight (Twilight, #1)",Stephenie Meyer,Young Adult|Fantasy|Romance|Paranormal|Vampire...,0.029020,3.58,3.5,4281268,97991,About three things I was absolutely positive.F...,to-read fantasy favorites currently-reading yo...,Twilight Stephenie-Meyer Young-Adult Fantasy R...,About three things I was absolutely positive.F...,456191,436802,793319,875073,1355439,NaN,Paperback,2005.0,en-US,498,4,226,41865,9.78032E+12,316015849,9.780316e+12,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443l...,https://images.gr-assets.com/books/1361039443s...
3,4,To Kill a Mockingbird,-,To Kill a Mockingbird,Harper Lee,Classics|Fiction|Historical|Historical Fiction...,0.231059,4.27,4.0,3745197,79450,The unforgettable novel of a childhood in a sl...,to-read favorites currently-reading young-adul...,To-Kill-a-Mockingbird Harper-Lee Classics Fict...,The unforgettable novel of a childhood in a sl...,60427,117415,446835,1001952,1714267,50th Anniversary,Paperback,1960.0,eng,324,3,487,2657,9.78006E+12,61120081,9.780061e+12,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680l...,https://images.gr-assets.com/books/1361975680s...
4,5,The Great Gatsby,-,The Great Gatsby,F. Scott Fitzgerald,Classics|Fiction|Academic|School|Literature|Hi...,0.049427,3.90,3.5,3141842,56953,Alternate Cover Edition ISBN: 0743273567 (ISBN...,to-read favorites currently-reading young-adul...,The-Great-Gatsby F.-Scott-Fitzgerald Classics ...,Alternate Cover Edition ISBN: 0743273567 (ISBN...,86236,197621,606158,936012,947718,US / CAN,Paperback,1925.0,eng,180,1,1356,4671,NaN,743273567,9.780743e+12,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560l...,https://images.gr-assets.com/books/1490528560s...
